### Ingesta del archivo "genre.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
v_environment

####Paso1 - Leer el archivo CSV usando DataFrameReader Spark

In [0]:
#importo los tipos de datos y estructuras
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
genre_schema = StructType(fields=[
    StructField("genreId", IntegerType(), False),
    StructField("genreName", StringType(), True)
])

In [0]:
genre_df = spark.read \
        .schema(genre_schema) \
        .option("header", True) \
        .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

In [0]:
display(genre_df)

####Paso2 - Cambiar nombre de las columnas

In [0]:
genre_renamed_df = genre_df.withColumnsRenamed({"genreId": "genre_id", "genreName": "genre_name"})

In [0]:
display(genre_renamed_df)

####Paso3 - Agragar columnas al DF

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = add_ingestion_dat(genre_renamed_df) \
                .withColumn("environment", lit(v_environment)) \
                .withColumn("file_date", lit(v_file_date))

In [0]:
display(genre_final_df)

####Paso5 - Escribir datos en el DataLake en formato Parquet

In [0]:
#overwrite_partition(genre_final_df, "movie_silver", "genre", "file_date")

In [0]:

genre_final_df.write.mode("append").partitionBy("file_date").format("delta").saveAsTable("movie_silver.genre")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.genre
group by file_date;

In [0]:
#genre_final_df.write.mode("append").partitionBy("file_date").parquet(f"{silver_folder_path}/genre")

In [0]:
%fs
ls /mnt/mymoviehistory/silver/genre

In [0]:
display(spark.read.format("delta").load(f"{silver_folder_path}/genre/"))



In [0]:
dbutils.notebook.exit("Success")